In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import optimize
import math

In [2]:
YEAR = 52
IR   = [0.0122, 0.0198, 0.0274, 0.0302, 0.035]
dt = 1/YEAR
ALPHA = 0.8
SIGMA = 1.2
h_0   = 0.0083696
R     = 0.25
N_SIM = 10000

In [3]:
IR_curve = [np.exp(-IR[1] * i / YEAR) for i in range(YEAR)] + \
           [np.exp(-IR[1]-IR[2] * i / YEAR) for i in range(YEAR)] + \
           [np.exp(-IR[1]-IR[2]-IR[3] * i / YEAR) for i in range(YEAR)] + \
           [np.exp(-IR[1]-IR[2]-IR[3]-IR[4] * i / YEAR) for i in range(YEAR)]

Z = np.random.normal(size=(3 * YEAR, N_SIM))
THETAS = [0.011852293096075071, 0.009040766594021893, 0.012600900100544096]
THRESHOLD = 0.00448

In [4]:
def discount(t, max_len):
    return IR_curve[t] if t < max_len else 0

def value(t, T, spread, R):
    n_payments = t // YEAR + 1
    return (1 - R) * discount(t, T) - sum([discount(i * YEAR, T) * spread for i in range(n_payments)])

def simulate(Z, alpha, sigma, h_0, thetas): # thetas = [theta_1, theta_2]
    # we have existing thetas, simulate for one more year
    _, N = Z.shape
    T = len(thetas) * YEAR
    results = np.empty([T, N])
    h_t  = np.full(N, h_0)
    for i, theta in enumerate(thetas):
        for j in range(YEAR):
            t = i * YEAR + j
            results[t, :] = h_t + alpha * (theta - h_t) * dt + sigma * h_t * math.sqrt(dt) * Z[t, :]
            h_t = results[t, :]
    return results

def sim_defaults(results):
    T, N = results.shape
    bern = np.random.binomial(1, results*dt)
    def default_fn(arr):
        out = np.argmax(arr)
        if out == 0 and arr[0] == 0:
            return len(arr)
        else: return out
    return np.apply_along_axis(default_fn, 0, bern), T

def rn_pricing(defaults, T, spread, R):
    return np.array(list(map(lambda t: value(t, T, spread, R), defaults)))

In [5]:
# this gives a T x N matrix of simulated hazard rates
results = simulate(Z, ALPHA, SIGMA, h_0, THETAS)

# this turn hazard rates into actual default timesteps
# note that if the output is exactly 3 * YEAR = 156, no default occured
defaults, T = sim_defaults(results)

# if you want to match a spread to the simulated defaults, do this
spread = 0.0045
pricing = rn_pricing(defaults, T, spread, R)

# calibrate such that pricing is 0

In [6]:
results[:10,:5]

array([[0.00892958, 0.00803796, 0.00950431, 0.00825475, 0.00725899],
       [0.01011166, 0.00790212, 0.00982899, 0.00975852, 0.00715546],
       [0.00652109, 0.00633581, 0.00952034, 0.00925546, 0.00776209],
       [0.00686948, 0.00677091, 0.01028746, 0.00793392, 0.00766905],
       [0.00658079, 0.00597369, 0.00878542, 0.00752967, 0.00578509],
       [0.00642281, 0.0063853 , 0.01199456, 0.00759609, 0.00624467],
       [0.00700541, 0.00763066, 0.01063758, 0.00601403, 0.00795553],
       [0.00821646, 0.00530037, 0.01069456, 0.00670952, 0.00872765],
       [0.00782892, 0.00554114, 0.00575345, 0.00867804, 0.00792654],
       [0.00802931, 0.00457781, 0.00613666, 0.00934055, 0.00786277]])

In [7]:
defaults[:5]

array([156, 156, 156, 156, 156])

In [8]:
pricing[:5]

array([-0.01320431, -0.01320431, -0.01320431, -0.01320431, -0.01320431])